# Colorization

Authors:

- Rohan Rele (rsr132)
- Aakash Raman (abr103)
- Alex Eng (ame136)
- Adarsh Patel (aap237)

This project was completed for Professor Wes Cowan's Fall 2019 offering of the CS 520: Intro to Artificial Intelligence course, taught at Rutgers University, New Brunswick.

# Problem Statement

# Process Representation

## Coloring process

## Feature spaces

## Maps

# Training Data

## Pre-processing

# Model: Convolutional Neural Network

## Intuition

## CNN architecture

## CNN implementation

# Model Evaluation

## Numerical error: loss function

## Perceptual error

# Model Training

## Stochastic gradient descent

## Back propagation

## Performance complexity

## Overfitting

# CNN Model Trials

# CNN Model Assessment

## Results analysis

## Colorization errors

## Brand new input data

## Future directions

# Bonus: Reconstructing damaged images